In [266]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyClassifier


from sklearn.metrics import confusion_matrix, plot_confusion_matrix,\
    precision_score, recall_score, accuracy_score, f1_score, log_loss,\
    roc_curve, roc_auc_score, classification_report

In [267]:
df = pd.read_csv("df_train.csv", low_memory = False)

In [268]:
df.head(2)

,Unnamed: 0,Ticket Number,Violation Date,Issuing Agency,Respondent First Name,Respondent Last Name,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),...,Charge #2: Code Section,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,complete violation location,complete respondent location,Hearing Result
0,545274,0162420262,2012-08-30,POLICE DEPARTMENT,RONALD,BARONE,BROOKLYN,NaN,NaN,1711,...,A.C. 24-238 A,IMPROPER AUDIBLE BURGLAR ALARM WITH NO AUTOMAT...,280.0,NaN,NaN,NaN,NaN,BROOKLYN 1711 EAST 33 STREET BROOKLYN 11234 ...,BROOKLYN 21 SEBA AVENUE BROOKLYN 11229 NEW YORK,2
1,25788,0112647160,2003-11-23,POLICE DEPARTMENT,JOHN,PEZZULLO,BROOKLYN,989.0,42.0,256,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BROOKLYN 989.0 42.0 256 7 AVENUE BROOKLYN 1121...,BROOKLYN 359 9 AVE BROOKLYN 11215 NEW YORK,0


In [269]:
df = df.drop("Unnamed: 0", axis=1)

In [270]:
def fill_na(column):
    
    for x in column:
        df[x].fillna('UNKNOWN', inplace=True)
    
    return df.head(2)

In [271]:
column_names = list(df.columns)

In [272]:
fill_na(column_names)

,Ticket Number,Violation Date,Issuing Agency,Respondent First Name,Respondent Last Name,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),Violation Location (Street Name),...,Charge #2: Code Section,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,complete violation location,complete respondent location,Hearing Result
0,0162420262,2012-08-30,POLICE DEPARTMENT,RONALD,BARONE,BROOKLYN,UNKNOWN,UNKNOWN,1711,EAST 33 STREET,...,A.C. 24-238 A,IMPROPER AUDIBLE BURGLAR ALARM WITH NO AUTOMAT...,280.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 1711 EAST 33 STREET BROOKLYN 11234 ...,BROOKLYN 21 SEBA AVENUE BROOKLYN 11229 NEW YORK,2
1,0112647160,2003-11-23,POLICE DEPARTMENT,JOHN,PEZZULLO,BROOKLYN,989.0,42.0,256,7 AVENUE,...,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 989.0 42.0 256 7 AVENUE BROOKLYN 1121...,BROOKLYN 359 9 AVE BROOKLYN 11215 NEW YORK,0


In [273]:
df["Charge #2: Infraction Amount"].value_counts().head(3)

UNKNOWN    209908
0.0          2447
1000.0        157
Name: Charge #2: Infraction Amount, dtype: int64

In [274]:
#to match 

df["Respondent Address (Zip Code)"].value_counts(normalize=True)

11207    0.029952
11368    0.025825
10466    0.025820
10457    0.023809
10456    0.023218
           ...   
11703    0.000005
10385    0.000005
10066    0.000005
12182    0.000005
11167    0.000005
Name: Respondent Address (Zip Code), Length: 1108, dtype: float64

### load zip code dataframe to add neighborhood level data to the original data

In [275]:
zip_code = pd.read_csv("Demographic_Statistics_By_Zip_Code.csv", low_memory = False)

In [29]:
zip_code.loc[zip_code["JURISDICTION NAME"] == 12783]

,JURISDICTION NAME,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,...,COUNT CITIZEN STATUS TOTAL,PERCENT CITIZEN STATUS TOTAL,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL
229,12783,201,66,0.33,135,0.67,0,0,201,100,...,201,100,77,0.38,124,0.62,0,0,201,100


In [30]:
merged_df = pd.merge(df, zip_code, left_on = "Respondent Address (Zip Code)", right_on = "JURISDICTION NAME", how="inner")

In [31]:
merged_df['Ticket Number'].is_unique

True

In [32]:
pd.set_option('display.max_columns', 2000000000)
merged_df.iloc[277360:]

,Ticket Number,Violation Date,Issuing Agency,Respondent First Name,Respondent Last Name,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),Violation Location (Street Name),Violation Location (City),Violation Location (Zip Code),Violation Location (State Name),Respondent Address (Borough),Respondent Address (House #),Respondent Address (Street Name),Respondent Address (City),Respondent Address (Zip Code),Respondent Address (State Name),Decision Location (Borough),Penalty Imposed,Paid Amount,Charge #1: Code,Charge #1: Code Section,Charge #1: Code Description,Charge #1: Infraction Amount,Charge #2: Code,Charge #2: Code Section,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,complete violation location,complete respondent location,Hearing Result,JURISDICTION NAME,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,COUNT PACIFIC ISLANDER,PERCENT PACIFIC ISLANDER,COUNT HISPANIC LATINO,PERCENT HISPANIC LATINO,COUNT AMERICAN INDIAN,PERCENT AMERICAN INDIAN,COUNT ASIAN NON HISPANIC,PERCENT ASIAN NON HISPANIC,COUNT WHITE NON HISPANIC,PERCENT WHITE NON HISPANIC,COUNT BLACK NON HISPANIC,PERCENT BLACK NON HISPANIC,COUNT OTHER ETHNICITY,PERCENT OTHER ETHNICITY,COUNT ETHNICITY UNKNOWN,PERCENT ETHNICITY UNKNOWN,COUNT ETHNICITY TOTAL,PERCENT ETHNICITY TOTAL,COUNT PERMANENT RESIDENT ALIEN,PERCENT PERMANENT RESIDENT ALIEN,COUNT US CITIZEN,PERCENT US CITIZEN,COUNT OTHER CITIZEN STATUS,PERCENT OTHER CITIZEN STATUS,COUNT CITIZEN STATUS UNKNOWN,PERCENT CITIZEN STATUS UNKNOWN,COUNT CITIZEN STATUS TOTAL,PERCENT CITIZEN STATUS TOTAL,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL


In [33]:
#moved the target column to the first for easier visual

first_column = merged_df.pop('Hearing Result')
merged_df.insert(0, 'Hearing Result', first_column)
merged_df.head(2)

,Hearing Result,Ticket Number,Violation Date,Issuing Agency,Respondent First Name,Respondent Last Name,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),Violation Location (Street Name),Violation Location (City),Violation Location (Zip Code),Violation Location (State Name),Respondent Address (Borough),Respondent Address (House #),Respondent Address (Street Name),Respondent Address (City),Respondent Address (Zip Code),Respondent Address (State Name),Decision Location (Borough),Penalty Imposed,Paid Amount,Charge #1: Code,Charge #1: Code Section,Charge #1: Code Description,Charge #1: Infraction Amount,Charge #2: Code,Charge #2: Code Section,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,complete violation location,complete respondent location,JURISDICTION NAME,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,COUNT PACIFIC ISLANDER,PERCENT PACIFIC ISLANDER,COUNT HISPANIC LATINO,PERCENT HISPANIC LATINO,COUNT AMERICAN INDIAN,PERCENT AMERICAN INDIAN,COUNT ASIAN NON HISPANIC,PERCENT ASIAN NON HISPANIC,COUNT WHITE NON HISPANIC,PERCENT WHITE NON HISPANIC,COUNT BLACK NON HISPANIC,PERCENT BLACK NON HISPANIC,COUNT OTHER ETHNICITY,PERCENT OTHER ETHNICITY,COUNT ETHNICITY UNKNOWN,PERCENT ETHNICITY UNKNOWN,COUNT ETHNICITY TOTAL,PERCENT ETHNICITY TOTAL,COUNT PERMANENT RESIDENT ALIEN,PERCENT PERMANENT RESIDENT ALIEN,COUNT US CITIZEN,PERCENT US CITIZEN,COUNT OTHER CITIZEN STATUS,PERCENT OTHER CITIZEN STATUS,COUNT CITIZEN STATUS UNKNOWN,PERCENT CITIZEN STATUS UNKNOWN,COUNT CITIZEN STATUS TOTAL,PERCENT CITIZEN STATUS TOTAL,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL
0,2,0162420262,2012-08-30,POLICE DEPARTMENT,RONALD,BARONE,BROOKLYN,UNKNOWN,UNKNOWN,1711,EAST 33 STREET,BROOKLYN,11234,NEW YORK,BROOKLYN,21,SEBA AVENUE,BROOKLYN,11229,NEW YORK,SAU: MANH,0.0,0.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,AN68,A.C. 24-238 A,IMPROPER AUDIBLE BURGLAR ALARM WITH NO AUTOMAT...,280.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 1711 EAST 33 STREET BROOKLYN 11234 ...,BROOKLYN 21 SEBA AVENUE BROOKLYN 11229 NEW YORK,11229,52,32,0.62,20,0.38,0,0,52,100,0,0.0,2,0.04,0,0.0,5,0.1,39,0.75,2,0.04,3,0.06,1,0.02,52,100,3,0.06,49,0.94,0,0.0,0,0,52,100,5,0.1,47,0.9,0,0,52,100
1,0,0176434684,2010-09-10,POLICE DEPARTMENT,KONSTANTIN,TSIPNYATOV,MANHATTAN,1016.0,36.0,1515,BROADWAY,NEW YORK,10036,NEW YORK,BROOKLYN,1815,EAST 17 STREET,BROOKLYN,11229,NEW YORK,UNKNOWN,1000.0,0.0,AG21,20-465.1,VENDING AT TIMES PLACES RESTRICTED BY RULE OF ...,100.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,MANHATTAN 1016.0 36.0 1515 BROADWAY NEW YORK 1...,BROOKLYN 1815 EAST 17 STREET BROOKLYN 11229 NE...,11229,52,32,0.62,20,0.38,0,0,52,100,0,0.0,2,0.04,0,0.0,5,0.1,39,0.75,2,0.04,3,0.06,1,0.02,52,100,3,0.06,49,0.94,0,0.0,0,0,52,100,5,0.1,47,0.9,0,0,52,100


In [34]:
df["Hearing Result"].value_counts(normalize=True)

0    0.429200
1    0.273505
3    0.150786
2    0.146509
Name: Hearing Result, dtype: float64

### for the respondents, some are individuals and some are commerical entities as indicated on some rows as "LLC." Therefore, need to create a separate column labeling whether the respondent is a person or otherwise

In [35]:
# pd.set_option('display.max_rows', 1000000000)
merged_df["Respondent First Name"].value_counts()

UNKNOWN      70502
MARIA         1438
JOSE          1358
LLC           1336
MOHAMED        974
             ...  
ADMEIR S         1
YANA             1
YORTY            1
PHYLLIS J        1
GUO QING         1
Name: Respondent First Name, Length: 33344, dtype: int64

In [36]:
merged_df.shape

(208022, 83)

In [37]:
key_words_first_name = ["INC", "CORP", "MANAGEMENT","BUS SERVICE AND TOUR", 
"SCIENCES DIVISION",
"HOUSING DEVELOPMENT"]  

In [38]:
key_words_last_name = ["INC", "CORP", "MANAGEMENT","FIRST HOME PROPERTIES",
"COR",
"3 NYC",
"HPENY HOUSING DEVELOPMENT FUND",
"RT HUDSON ELEMENTARY SCHOOL",
"DEVELOPMENT CO",
"HOLDING CO",
"BANANA KELLY HSG DEVE",
"AQUA PROPERTIES",
"THE BROOKLYN UNION GAS CO",
"VANDERBILT MORTGAGE AND FINANC",
"AMERICAN BROKERS CONDUIT",
"CMI BUSINESS FURNITURE",
"FRIENDS LAND DEVELOP",
"HARBOR VIEW PROP LTD",
"INGERSOLL TENANT ASSOC",
"THE BROOKLYN UNION GAS COMPANY ",
"PLAZA CONSTRUCTION",
"AUTO AUCTION"
"FIRST HOME PROP",
"1046 WASHINGTON AVE HDFC",
"DIEGO BEEKMAN MUTUAL HOUSING A",
"REV MANAGEMENT",
"LANDSLIDE PROPERTIES",
"NEIGHBORHOOD RESTORE HOUSING D",
"HTB ENTERPRISES LTD",
"ALLIANCE OF INDIVIDUA",
"WJR PROPERTIES INC",
"WJR PROPERTIES INC",
"KEYSPAN ENERGY DELIVERY NYC",
"RLTY",
"FIRST UNITED MORTGAGE BANKING",
"ASSET PLUSS MANAGEMENT SERVICE",
"KEYSPAN ENERGY DELIVERY N Y C",
"WELLS FARGO HOME MORT",
"ALLIANCE OF INDIVIDUAL",
"NEIGHBORHOOD RESTORE HDFC",
"WILMINGTON SAVINGS FUND SOCIET",
"YOUNG ISRAEL OF AVENUE K",
"FREMONT INVESTMENT LOAN",
"BELL ATLANTIC",
"EM ESS PETROLEUM CORP",
"PI CONSTRUCTION SERVICE INC",
"US BANK NATIONAL ASSOCIATION",
"CONKLIN MGMT CO",
"CON EDISON",
"CONSOLIDATED EDISON",
"EMPIRE CITY SUBWAY",
"DEUTSCHE BANK NATIONAL TRUST C",
"NATIONAL GRID",
"CONTACT HOLDINGS CORP",
"U S BANK NATIONAL ASSOCIATION",
"G G ASSOCIATES",
"WELLS FARGO BANK",
"LUCKY SEAFOOD",
"AGENT OWNER",
"FEDERAL NATIONAL MORTGAGE ASSO",
"AMENCAN HOME MORTGAGE",
"HOMESIDE LENDING",
"HSBC BANK USA",
"HSBC BANK USA NA",
"HIGH STATE RLTY CORP",
"NYC HOUSING AUTHORITY",
"PLAZA CONSTRUCTION CORP",
"EASY STREET PLUMBING INC",
"1249 WEBSTER AVE RLTY",
"DEVELOP", "BANK", "RESOURCES", "SERVICES", "LLC", "SCHOOL", "HOME","NATIONAL GRID","SAM CONEY ISLAND LLC"
                    "ALL PHASE PLUMBING CORP","ERCAT REALTY CORP"]

In [39]:
merged_df['Respondent Last Name'] = merged_df['Respondent Last Name'].astype(str)

In [40]:
# def word_checker(sentence):
#     if any(word in key_words_last_name for word in sentence.lower().split()):
#         return 'Not Person'
#     else:
#         return 'Person'

In [41]:
# merged_df['Respondent Status'] = merged_df['Respondent Last Name'].apply(word_checker)  

In [42]:
def get_word(my_string):
    for word in key_words_last_name:
        for x in merged_df["Respondent Last Name"]:
            if word.lower() in my_string.lower():
                return "Not Person"
            else:
                return "Person"

In [43]:
merged_df["Respondent Status"]= merged_df["Respondent Last Name"].apply(get_word)

In [44]:
merged_df.sample(2)

,Hearing Result,Ticket Number,Violation Date,Issuing Agency,Respondent First Name,Respondent Last Name,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),Violation Location (Street Name),Violation Location (City),Violation Location (Zip Code),Violation Location (State Name),Respondent Address (Borough),Respondent Address (House #),Respondent Address (Street Name),Respondent Address (City),Respondent Address (Zip Code),Respondent Address (State Name),Decision Location (Borough),Penalty Imposed,Paid Amount,Charge #1: Code,Charge #1: Code Section,Charge #1: Code Description,Charge #1: Infraction Amount,Charge #2: Code,Charge #2: Code Section,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,complete violation location,complete respondent location,JURISDICTION NAME,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,COUNT PACIFIC ISLANDER,PERCENT PACIFIC ISLANDER,COUNT HISPANIC LATINO,PERCENT HISPANIC LATINO,COUNT AMERICAN INDIAN,PERCENT AMERICAN INDIAN,COUNT ASIAN NON HISPANIC,PERCENT ASIAN NON HISPANIC,COUNT WHITE NON HISPANIC,PERCENT WHITE NON HISPANIC,COUNT BLACK NON HISPANIC,PERCENT BLACK NON HISPANIC,COUNT OTHER ETHNICITY,PERCENT OTHER ETHNICITY,COUNT ETHNICITY UNKNOWN,PERCENT ETHNICITY UNKNOWN,COUNT ETHNICITY TOTAL,PERCENT ETHNICITY TOTAL,COUNT PERMANENT RESIDENT ALIEN,PERCENT PERMANENT RESIDENT ALIEN,COUNT US CITIZEN,PERCENT US CITIZEN,COUNT OTHER CITIZEN STATUS,PERCENT OTHER CITIZEN STATUS,COUNT CITIZEN STATUS UNKNOWN,PERCENT CITIZEN STATUS UNKNOWN,COUNT CITIZEN STATUS TOTAL,PERCENT CITIZEN STATUS TOTAL,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL,Respondent Status
2202,3,040623779M,2013-11-18,SANITATION POLICE,UNKNOWN,CARR JOHN,QUEENS,10002.0,22.0,95-32,150 STREET,JAMAICA,11435,NEW YORK,QUEENS,95-32,150 STREET,JAMAICA,11435,NEW YORK,UNKNOWN,300.0,280.0,AS9C,16-120 D,LOOSE RUBBISH - 2ND OFFENSE,100.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,QUEENS 10002.0 22.0 95-32 150 STREET JAMAICA 1...,QUEENS 95-32 150 STREET JAMAICA 11435 NEW YORK,11435,0,0,0.00,0,0.00,0,0,0,0,0,0.0,0,0.00,0,0.0,0,0.0,0,0.00,0,0.0,0,0.0,0,0.0,0,0,0,0.00,0,0.00,0,0.0,0,0,0,0,0,0.00,0,0.00,0,0,0,0,Person
77725,0,0134520110,2004-06-15,POLICE DEPARTMENT,JIE,QIU,MANHATTAN,196.0,9.0,276,CANAL STREET,MANHATTAN,10013,NEW YORK,MANHATTAN,160,MADISON STREET,MANHATTAN,10002,NEW YORK,UNKNOWN,250.0,UNKNOWN,AG02,A.C. 20-461(A),FAILURE TO CARRY AND EXHIBIT LICENSE ON DEMAND,100.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,MANHATTAN 196.0 9.0 276 CANAL STREET MANHATTAN...,MANHATTAN 160 MADISON STREET MANHATTAN 10002 N...,10002,35,19,0.54,16,0.46,0,0,35,100,0,0.0,1,0.03,0,0.0,28,0.8,6,0.17,0,0.0,0,0.0,0,0.0,35,100,2,0.06,33,0.94,0,0.0,0,0,35,100,2,0.06,33,0.94,0,0,35,100,Person


In [45]:
merged_df['Respondent Status'].value_counts()

Person        197595
Not Person     10427
Name: Respondent Status, dtype: int64

In [46]:
merged_df.loc[merged_df['Respondent Last Name'] == "NATIONAL GRID"]

,Hearing Result,Ticket Number,Violation Date,Issuing Agency,Respondent First Name,Respondent Last Name,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),Violation Location (Street Name),Violation Location (City),Violation Location (Zip Code),Violation Location (State Name),Respondent Address (Borough),Respondent Address (House #),Respondent Address (Street Name),Respondent Address (City),Respondent Address (Zip Code),Respondent Address (State Name),Decision Location (Borough),Penalty Imposed,Paid Amount,Charge #1: Code,Charge #1: Code Section,Charge #1: Code Description,Charge #1: Infraction Amount,Charge #2: Code,Charge #2: Code Section,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,complete violation location,complete respondent location,JURISDICTION NAME,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,COUNT PACIFIC ISLANDER,PERCENT PACIFIC ISLANDER,COUNT HISPANIC LATINO,PERCENT HISPANIC LATINO,COUNT AMERICAN INDIAN,PERCENT AMERICAN INDIAN,COUNT ASIAN NON HISPANIC,PERCENT ASIAN NON HISPANIC,COUNT WHITE NON HISPANIC,PERCENT WHITE NON HISPANIC,COUNT BLACK NON HISPANIC,PERCENT BLACK NON HISPANIC,COUNT OTHER ETHNICITY,PERCENT OTHER ETHNICITY,COUNT ETHNICITY UNKNOWN,PERCENT ETHNICITY UNKNOWN,COUNT ETHNICITY TOTAL,PERCENT ETHNICITY TOTAL,COUNT PERMANENT RESIDENT ALIEN,PERCENT PERMANENT RESIDENT ALIEN,COUNT US CITIZEN,PERCENT US CITIZEN,COUNT OTHER CITIZEN STATUS,PERCENT OTHER CITIZEN STATUS,COUNT CITIZEN STATUS UNKNOWN,PERCENT CITIZEN STATUS UNKNOWN,COUNT CITIZEN STATUS TOTAL,PERCENT CITIZEN STATUS TOTAL,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL,Respondent Status
58597,3,0182000693,2013-10-12,POLICE DEPT,UNKNOWN,NATIONAL GRID,BROOKLYN,UNKNOWN,UNKNOWN,136,SOUTH 4TH STREET,BROOKLYN,11211,NEW YORK,BROOKLYN,1,METROTECH CENTER,BROOKLYN,11201,NEW YORK,UNKNOWN,250.0,234.0,AD16,A.C. 19-122,SAND DIRT RUBBISH DEBRIS NOT REMOVED FROM SITE...,25000.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 136 SOUTH 4TH STREET BROOKLYN 11211...,BROOKLYN 1 METROTECH CENTER BROOKLYN 11201 NEW...,11201,11,6,0.55,5,0.45,0,0,11,100,0,0.0,2,0.18,0,0.0,1,0.09,1,0.09,5,0.45,2,0.18,0,0.0,11,99,0,0.0,11,1.0,0,0.0,0,0,11,100,2,0.18,9,0.82,0,0,11,100,Person
58607,1,0177039574,2010-08-09,NYPD TRANSPORT INTELLIGENCE DI,UNKNOWN,NATIONAL GRID,BROOKLYN,UNKNOWN,UNKNOWN,UNKNOWN,F O 1664 EASTERN PARKWAY BT,BROOKLYN,11233,NEW YORK,BROOKLYN,ONE,METROTECH CENTER,BROOKLYN,11201,NEW YORK,UNKNOWN,3600.0,1200.0,AD30,A.C. 19-102 II,FAILURE TO COMPLY WITH THE TERMS AND CONDITION...,1200.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN F O 1664 EASTERN PARKWAY BT BROOKL...,BROOKLYN ONE METROTECH CENTER BROOKLYN 11201 N...,11201,11,6,0.55,5,0.45,0,0,11,100,0,0.0,2,0.18,0,0.0,1,0.09,1,0.09,5,0.45,2,0.18,0,0.0,11,99,0,0.0,11,1.0,0,0.0,0,0,11,100,2,0.18,9,0.82,0,0,11,100,Person
58609,3,0182000583,2013-07-24,POLICE DEPT,UNKNOWN,NATIONAL GRID,BROOKLYN,3031.0,10.0,12,STAGG STREET,BROOKLYN,11206,NEW YORK,BROOKLYN,1,METROTECT CENTER,BROOKLYN,11201,NEW YORK,UNKNOWN,280.0,262.0,AD16,A.C. 19-122,SAND DIRT RUBBISH DEBRIS NOT REMOVED FROM SITE...,25000.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 3031.0 10.0 12 STAGG STREET BROOKLYN ...,BROOKLYN 1 METROTECT CENTER BROOKLYN 11201 NEW...,11201,11,6,0.55,5,0.45,0,0,11,100,0,0.0,2,0.18,0,0.0,1,0.09,1,0.09,5,0.45,2,0.18,0,0.0,11,99,0,0.0,11,1.0,0,0.0,0,0,11,100,2,0.18,9,0.82,0,0,11,100,Person
58610,1,0180233662,2012-04-03,NYPD TRANSPORT INTELLIGENCE DI,UNKNOWN,NATIONAL GRID

In [47]:
merged_df.drop("JURISDICTION NAME", axis=1, inplace=True)

In [48]:
merged_df.head(1)

,Hearing Result,Ticket Number,Violation Date,Issuing Agency,Respondent First Name,Respondent Last Name,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),Violation Location (Street Name),Violation Location (City),Violation Location (Zip Code),Violation Location (State Name),Respondent Address (Borough),Respondent Address (House #),Respondent Address (Street Name),Respondent Address (City),Respondent Address (Zip Code),Respondent Address (State Name),Decision Location (Borough),Penalty Imposed,Paid Amount,Charge #1: Code,Charge #1: Code Section,Charge #1: Code Description,Charge #1: Infraction Amount,Charge #2: Code,Charge #2: Code Section,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,complete violation location,complete respondent location,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,COUNT PACIFIC ISLANDER,PERCENT PACIFIC ISLANDER,COUNT HISPANIC LATINO,PERCENT HISPANIC LATINO,COUNT AMERICAN INDIAN,PERCENT AMERICAN INDIAN,COUNT ASIAN NON HISPANIC,PERCENT ASIAN NON HISPANIC,COUNT WHITE NON HISPANIC,PERCENT WHITE NON HISPANIC,COUNT BLACK NON HISPANIC,PERCENT BLACK NON HISPANIC,COUNT OTHER ETHNICITY,PERCENT OTHER ETHNICITY,COUNT ETHNICITY UNKNOWN,PERCENT ETHNICITY UNKNOWN,COUNT ETHNICITY TOTAL,PERCENT ETHNICITY TOTAL,COUNT PERMANENT RESIDENT ALIEN,PERCENT PERMANENT RESIDENT ALIEN,COUNT US CITIZEN,PERCENT US CITIZEN,COUNT OTHER CITIZEN STATUS,PERCENT OTHER CITIZEN STATUS,COUNT CITIZEN STATUS UNKNOWN,PERCENT CITIZEN STATUS UNKNOWN,COUNT CITIZEN STATUS TOTAL,PERCENT CITIZEN STATUS TOTAL,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL,Respondent Status
0,2,0162420262,2012-08-30,POLICE DEPARTMENT,RONALD,BARONE,BROOKLYN,UNKNOWN,UNKNOWN,1711,EAST 33 STREET,BROOKLYN,11234,NEW YORK,BROOKLYN,21,SEBA AVENUE,BROOKLYN,11229,NEW YORK,SAU: MANH,0.0,0.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,AN68,A.C. 24-238 A,IMPROPER AUDIBLE BURGLAR ALARM WITH NO AUTOMAT...,280.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 1711 EAST 33 STREET BROOKLYN 11234 ...,BROOKLYN 21 SEBA AVENUE BROOKLYN 11229 NEW YORK,52,32,0.62,20,0.38,0,0,52,100,0,0.0,2,0.04,0,0.0,5,0.1,39,0.75,2,0.04,3,0.06,1,0.02,52,100,3,0.06,49,0.94,0,0.0,0,0,52,100,5,0.1,47,0.9,0,0,52,100,Person


In [51]:
# pd.options.display.max_colwidth = 1000000
# pd.set_option('display.max_columns', 2000000000)
# pd.set_option('display.max_rows', 1000000000)
# pd.set_option('display.expand_frame_repr', True)

# FSM

just pulled in one feature b/c i need to do more work on the columns. 

In [50]:
dummy_model = DummyClassifier(strategy="most_frequent")
dummy_model.fit(df["Violation Location (City)"], df["Hearing Result"])
y_hat = dummy_model.predict(df)

In [54]:
acc = accuracy_score(df["Hearing Result"],y_hat)
macro_precision_score=precision_score(df["Hearing Result"], y_hat, average='macro')
micro_precision_score=precision_score(df["Hearing Result"] , y_hat, average='micro')
macro_recall_score=recall_score(df["Hearing Result"], y_hat, average='macro')
micro_recall_score=recall_score(df["Hearing Result"], y_hat, average='micro')

print('Accuracy Score: {}'.format(acc))
print('Macro Precision Score: {}'.format(macro_precision_score))
print('Micro Precision Score: {}'.format(micro_precision_score))
print('Macro Recall Score: {}'.format(macro_recall_score))
print('Micro Recall Score: {}'.format(micro_recall_score))

Accuracy Score: 0.4292004895823075
Macro Precision Score: 0.10730012239557687
Micro Precision Score: 0.4292004895823075
Macro Recall Score: 0.25
Micro Recall Score: 0.4292004895823075


/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Generate a column on neighborhood level income data (using Census data)

1. add a new column to the datafram with ZIP Code Tabulation Areas (ZCTAs)
2. bring in income data matching on ZCTAs
3. to download ZCTAS, will need to do web scrapping

In [177]:
nyc_zip_codes = ["10001",
"10002",
"10003",
"10004",
"10005",
"10006",
"10007",
"10009",
"10010",
"10011",
"10012",
"10013",
"10014",
"10015",
"10016",
"10017",
"10018",
"10019",
"10020",
"10021",
"10022",
"10023",
"10024",
"10025",
"10026",
"10027",
"10028",
"10029",
"10030",
"10031",
"10032",
"10033",
"10034",
"10035",
"10036",
"10037",
"10038",
"10039",
"10040",
"10041",
"10044",
"10045",
"10048",
"10055",
"10060",
"10069",
"10090",
"10095",
"10098",
"10099",
"10103",
"10104",
"10105",
"10106",
"10107",
"10110",
"10111",
"10112",
"10115",
"10118",
"10119",
"10120",
"10121",
"10122",
"10123",
"10128",
"10151",
"10152",
"10153",
"10154",
"10155",
"10158",
"10161",
"10162",
"10165",
"10166",
"10167",
"10168",
"10169",
"10170",
"10171",
"10172",
"10173",
"10174",
"10175",
"10176",
"10177",
"10178",
"10199",
"10270",
"10271",
"10278",
"10279",
"10280",
"10281",
"10282",
"10301",
"10302",
"10303",
"10304",
"10305",
"10306",
"10307",
"10308",
"10309",
"10310",
"10311",
"10312",
"10314",
"10451",
"10452",
"10453",
"10454",
"10455",
"10456",
"10457",
"10458",
"10459",
"10460",
"10461",
"10462",
"10463",
"10464",
"10465",
"10466",
"10467",
"10468",
"10469",
"10470",
"10471",
"10472",
"10473",
"10474",
"10475",
"11004",
"11101",
"11102",
"11103",
"11104",
"11105",
"11106",
"11109",
"11201",
"11203",
"11204",
"11205",
"11206",
"11207",
"11208",
"11209",
"11210",
"11211",
"11212",
"11213",
"11214",
"11215",
"11216",
"11217",
"11218",
"11219",
"11220",
"11221",
"11222",
"11223",
"11224",
"11225",
"11226",
"11228",
"11229",
"11230",
"11231",
"11232",
"11233",
"11234",
"11235",
"11236",
"11237",
"11238",
"11239",
"11241",
"11242",
"11243",
"11249",
"11252",
"11256",
"11351",
"11354",
"11355",
"11356",
"11357",
"11358",
"11359",
"11360",
"11361",
"11362",
"11363",
"11364",
"11365",
"11366",
"11367",
"11368",
"11369",
"11370",
"11371",
"11372",
"11373",
"11374",
"11375",
"11377",
"11378",
"11379",
"11385",
"11411",
"11412",
"11413",
"11414",
"11415",
"11416",
"11417",
"11418",
"11419",
"11420",
"11421",
"11422",
"11423",
"11426",
"11427",
"11428",
"11429",
"11430",
"11432",
"11433",
"11434",
"11435",
"11436",
"11691",
"11692",
"11693",
"11694",
"11697"]


In [363]:
# this dataset contains USPS zip_code and ZCTA zip code for several states. 
#It indicates whether some USPS and ZCTA willl match up and if it doesn't, the equivalence of that
ZiptoZcta_Crosswalk_2021 = pd.read_excel("ZiptoZcta_Crosswalk_2021.xlsx")
ZiptoZcta_Crosswalk_2021["ZIP_CODE"] = ZiptoZcta_Crosswalk_2021["ZIP_CODE"].astype(str)
df['Respondent Address (Zip Code)'] = df['Respondent Address (Zip Code)'].astype(str)

# only pull out zip codes that matches the nyc zipcode list above
ZiptoZcta_Crosswalk_2021["ZIP_CODE_NYC"] = np.where(ZiptoZcta_Crosswalk_2021["ZIP_CODE"].isin(nyc_zip_codes), "NYC", "Other")

In [364]:
ZiptoZcta_Crosswalk_2021["ZIP_CODE_NYC"].value_counts()

Other    40873
NYC        234
Name: ZIP_CODE_NYC, dtype: int64

In [365]:
ZiptoZcta_Crosswalk_2021_NYC = ZiptoZcta_Crosswalk_2021.loc[ZiptoZcta_Crosswalk_2021["ZIP_CODE_NYC"] == "NYC"]
ZiptoZcta_Crosswalk_2021_NYC

,ZIP_CODE,PO_NAME,STATE,ZIP_TYPE,ZCTA,zip_join_type,ZIP_CODE_NYC
0,10001,New York,NY,Zip Code Area,10001,Zip matches ZCTA,NYC
1,10002,New York,NY,Zip Code Area,10002,Zip matches ZCTA,NYC
3190,10003,New York,NY,Zip Code Area,10003,Zip matches ZCTA,NYC
3191,10004,New York,NY,Zip Code Area,10004,Zip matches ZCTA,NYC
3192,10005,New York,NY,Zip Code Area,10005,Zip matches ZCTA,NYC
3193,10006,New York,NY,Zip Code Area,10006,Zip matches ZCTA,NYC
3194,10007,New York,NY,Zip Code Area,10007,Zip matches ZCTA,NYC
3195,10009,New York,NY,Zip Code Area,10009,Zip matches ZCTA,NYC
3196,10010,New York,NY,Zip Code Area,10010,Zip matches ZCTA,NYC
3197,10011,New York,NY,Zip Code Area,10011,Zip matches ZCTA,NYC


In [366]:
ZiptoZcta_Crosswalk_2021_NYC.to_csv("NYC_Only_ZiptoZcta_Crosswalk_2021.csv")

In [367]:
ZiptoZcta_Crosswalk_2021_NYC

,ZIP_CODE,PO_NAME,STATE,ZIP_TYPE,ZCTA,zip_join_type,ZIP_CODE_NYC
0,10001,New York,NY,Zip Code Area,10001,Zip matches ZCTA,NYC
1,10002,New York,NY,Zip Code Area,10002,Zip matches ZCTA,NYC
3190,10003,New York,NY,Zip Code Area,10003,Zip matches ZCTA,NYC
3191,10004,New York,NY,Zip Code Area,10004,Zip matches ZCTA,NYC
3192,10005,New York,NY,Zip Code Area,10005,Zip matches ZCTA,NYC
3193,10006,New York,NY,Zip Code Area,10006,Zip matches ZCTA,NYC
3194,10007,New York,NY,Zip Code Area,10007,Zip matches ZCTA,NYC
3195,10009,New York,NY,Zip Code Area,10009,Zip matches ZCTA,NYC
3196,10010,New York,NY,Zip Code Area,10010,Zip matches ZCTA,NYC
3197,10011,New York,NY,Zip Code Area,10011,Zip matches ZCTA,NYC


In [348]:
ZiptoZcta_Crosswalk_2021_NYC_dict = dict(zip(ZiptoZcta_Crosswalk_2021_NYC.ZIP_CODE, ZiptoZcta_Crosswalk_2021_NYC.ZCTA))

In [349]:
ZiptoZcta_Crosswalk_2021_NYC_dict

{'10001': '10001',
 '10002': '10002',
 '10003': '10003',
 '10004': '10004',
 '10005': '10005',
 '10006': '10006',
 '10007': '10007',
 '10009': '10009',
 '10010': '10010',
 '10011': '10011',
 '10012': '10012',
 '10013': '10013',
 '10014': '10014',
 '10016': '10016',
 '10017': '10017',
 '10018': '10018',
 '10019': '10019',
 '10020': '10020',
 '10021': '10021',
 '10022': '10022',
 '10023': '10023',
 '10024': '10024',
 '10025': '10025',
 '10026': '10026',
 '10027': '10027',
 '10028': '10028',
 '10029': '10029',
 '10030': '10030',
 '10031': '10031',
 '10032': '10032',
 '10033': '10033',
 '10034': '10034',
 '10035': '10035',
 '10036': '10036',
 '10037': '10037',
 '10038': '10038',
 '10039': '10039',
 '10040': '10040',
 '10044': '10044',
 '10069': '10069',
 '10103': '10103',
 '10110': '10110',
 '10111': '10111',
 '10112': '10112',
 '10115': '10115',
 '10119': '10119',
 '10128': '10128',
 '10152': '10152',
 '10153': '10153',
 '10154': '10154',
 '10162': '10162',
 '10165': '10165',
 '10167': '1

In [350]:
df['Respondent ZCTA'] = df['Respondent Address (Zip Code)'].map(ZiptoZcta_Crosswalk_2021_NYC_dict)

In [357]:
df["Respondent ZCTA"].isnull().value_counts()

False    208967
True       4276
Name: Respondent ZCTA, dtype: int64

In [360]:
df.sample(50)

,Ticket Number,Violation Date,Issuing Agency,Respondent First Name,Respondent Last Name,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),Violation Location (Street Name),Violation Location (City),Violation Location (Zip Code),Violation Location (State Name),Respondent Address (Borough),Respondent Address (House #),Respondent Address (Street Name),Respondent Address (City),Respondent Address (Zip Code),Respondent Address (State Name),Decision Location (Borough),Penalty Imposed,Paid Amount,Charge #1: Code,Charge #1: Code Section,Charge #1: Code Description,Charge #1: Infraction Amount,Charge #2: Code,Charge #2: Code Section,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,complete violation location,complete respondent location,Hearing Result,Respondent ZCTA
180139,0124811116,2003-01-25,POLICE DEPARTMENT,XAW,MING,MANHATTAN,1013.0,12.0,234,WEST 42 STREET,MANHATTAN,10036,NEW YORK,QUEENS,42-49,79 STREET,QUEENS,11373,NEW YORK,UNKNOWN,50.0,UNKNOWN,AG66,6 RCNY 2-307 C,FAILURE TO OFFER RECEIPT FOR PURCHASE,100.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,MANHATTAN 1013.0 12.0 234 WEST 42 STREET MANHA...,QUEENS 42-49 79 STREET QUEENS 11373 NEW YORK,0,11373
201724,0104503998,1999-11-08,POLICE DEPARTMENT,LOO,QURTANO,QUEENS,10155.0,1.0,165-02,JAMAICA AVENUE,QNS,11432,NEW YORK,NOT NYC,152,FEDRICK AVE,FLORAL PARK,11003,NEW YORK,UNKNOWN,350.0,0.0,AN06,24-220 A,UNREASONABLE NOISE SOUND REPRODUCTION DEVICE,140.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,QUEENS 10155.0 1.0 165-02 JAMAICA AVENUE QNS 1...,NOT NYC 152 FEDRICK AVE FLORAL PARK 11003 NEW ...,0,NaN
158963,040424276Z,2009-07-15,SANITATION POLICE,DANIEL,MOORE,BROOKLYN,7895.0,11.0,2233,UTICA AVENUE,BROOKLYN,11234,NEW YORK,BROOKLYN,227,SULLIVAN PLACE,BROOKLYN,11225,NEW YORK,UNKNOWN,150.0,0.0,AS24,A.C. 16-122 C,DISABLED VEHICLE,100.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 7895.0 11.0 2233 UTICA AVENUE BROOKLY...,BROOKLYN 227 SULLIVAN PLACE BROOKLYN 11225 NEW...,0,11225
160328,040545723M,2012-08-24,SANITATION POLICE,UNKNOWN,DAUGHTRY JAMES W,BROOKLYN,3758.0,27.0,334,BRADFORD STREET,BROOKLYN,11207,NEW YORK,BROOKLYN,334,BRADFORD STREET,BROOKLYN,11207,NEW YORK,UNKNOWN,300.0,0.0,AS97,16-118 2,"DIRTY SIDEWALK,FAIL TO CLEAN 18 INTO STREET,SI...",250.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 3758.0 27.0 334 BRADFORD STREET BROOK...,BROOKLYN 334 BRADFORD STREET BROOKLYN 11207 NE...,0,11207
86473,0162156886,2008-01-10,NYPD TRANSPORT INTELLIGENCE DI,UNKNOWN,KISKA GROUP LTD,BROOKLYN,2298.0,31.0,165,NORTH 10 STREET,BROOKLYN,11211,NEW YORK,BROOKLYN,181,NORTH 11 STREET,BROOKLYN,11211,NEW YORK,BROOKLYN,250.0,250.0,AD17,A.C. 19-123,NO STREET PROTECTION UNDER COMMERCIAL REFUSE C...,250.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 2298.0 31.0 165 NORTH 10 STREET BROOK...,BROOKLYN 181 NORTH 11 STREET BROOKLYN 11211 NE...,3,11211
13957,0137577936,2003-09-26,SANITATION POLICE,DASKAL,MENDEL,BROOKLYN,1753.0,40.0,180,NOSTRAND AVENUE,BROOKLYN,11205,NEW YORK,BROOKLYN,180,NOSTRAND AVENUE,BROOKLYN,11205,NEW YORK,UNKNOWN,300.0,UNKNOWN,AS06,16-118 2,DIRTY SIDEWALK,100.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 1753.0 40.0 180 NOSTRAND AVENUE BROOK...,BROOKLYN 180 NOSTRAND AVENUE BROOKLYN 11205 NE...,0,11205
149828,042552464P,2012-12-19,SANITATION POLICE,DELVI,LIBERATO,BRONX,5263.0,244.0,3406,WRIGHT AVENUE,BRONX,10475,NEW YORK,BRONX,3406,WRIGHT AVENUE,BRONX,10475,NEW YORK,UNKNOWN,300.0,305.0,AS6M,A.C. 16-118 2 A,DIRTY AREA,100.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BRONX 5263.0 244.0 3406 WRIGHT AVENUE BRONX 10...,BRONX 3406 WRIGHT AVENUE BRONX 10475 NEW YORK,3,10475
167779,043301162J,2009-08-11,SANITATION POLICE,UNKNOWN,MERRY GO AROUND,MANHATTAN,476.0,57.0

In [257]:
ZiptoZcta_Crosswalk_2021_NYC['zip_join_type'].value_counts()

Zip matches ZCTA        205
Spatial join to ZCTA     29
Name: zip_join_type, dtype: int64

In [279]:
# get a list of the zipcode that matches ZCTA and map it onto the new column
nyc_zip_code_list = list(ZiptoZcta_Crosswalk_2021["ZIP_CODE"][ZiptoZcta_Crosswalk_2021['zip_join_type'] == "Zip matches ZCTA"])

In [278]:
df['Respondent Address (Zip Code)']

0         11229
1         11215
2         11435
3         11208
4         11367
          ...  
213238    11229
213239    10466
213240    11224
213241    11208
213242    11373
Name: Respondent Address (Zip Code), Length: 213243, dtype: int64

In [335]:
df['Respondent ZCTA'] = np.where(df['Respondent Address (Zip Code)'].isin(nyc_zip_code_list), df['Respondent Address (Zip Code)'], "Not Matched")

In [302]:
pd.set_option('display.max_columns', 100000000000000)

In [304]:
# check to make sure it works
df.sample(20)

,Ticket Number,Violation Date,Issuing Agency,Respondent First Name,Respondent Last Name,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),Violation Location (Street Name),Violation Location (City),Violation Location (Zip Code),Violation Location (State Name),Respondent Address (Borough),Respondent Address (House #),Respondent Address (Street Name),Respondent Address (City),Respondent Address (Zip Code),Respondent Address (State Name),Decision Location (Borough),Penalty Imposed,Paid Amount,Charge #1: Code,Charge #1: Code Section,Charge #1: Code Description,Charge #1: Infraction Amount,Charge #2: Code,Charge #2: Code Section,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,complete violation location,complete respondent location,Hearing Result,Respondent ZCTA
22059,0201236283,2020-12-06,POLICE DEPARTMENT,ARIEL,MASOLLER,BROOKLYN,4314.0,1.0,792,NEW LOTS AVENUE,BROOKLYN,11208,NEW YORK,BROOKLYN,387A,53 STREET,BROOKLYN,11220,NEW YORK,UNKNOWN,25.0,0.0,AX25,AC 10-125,OPEN CONTAINER CONSUMPTION OF ALCOHOL ON STREETS,25.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 4314.0 1.0 792 NEW LOTS AVENUE BROOKL...,BROOKLYN 387A 53 STREET BROOKLYN 11220 NEW YORK,1,11220
28204,0151843735,2006-03-17,SANITATION POLICE,UNKNOWN,YK REALTY INC,BROOKLYN,UNKNOWN,UNKNOWN,3030,BRIGHTON 3 STREET,BROOKLYN,11235,NEW YORK,BROOKLYN,3030,BRIGHTON 3 STREET,BROOKLYN,11235,NEW YORK,UNKNOWN,300.0,369.0,AS06,16-118 2,DIRTY SIDEWALK,100.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 3030 BRIGHTON 3 STREET BROOKLYN 112...,BROOKLYN 3030 BRIGHTON 3 STREET BROOKLYN 11235...,1,11235
51692,0134097600,2014-07-15,POLICE DEPARTMENT,JOSEPH,PECORA,QUEENS,UNKNOWN,UNKNOWN,56-45,MAIN STREET,FLUSHING,11355,UNKNOWN,QUEENS,53-51,205 STREET,OAKLAND GARDENS,11364,NEW YORK,UNKNOWN,250.0,250.0,AF19,17-315 F,VIOLATION OF PARKING RULES AND REGULATIONS,0.0,UNKNOWN,UNKNOWN,UNKNOWN,0.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,QUEENS 56-45 MAIN STREET FLUSHING 11355,QUEENS 53-51 205 STREET OAKLAND GARDENS 11364 ...,1,11364
66459,0196445756,2019-01-01,POLICE DEPARTMENT,NIAKATE,MAMADOU,BROOKLYN,2969.0,14.0,492,SCHOLES STREET,BROOKLYN,11237,NEW YORK,MANHATTAN,235,WEST 48 STREET,NEW YORK,10036,NEW YORK,UNKNOWN,112.0,0.0,AS9I,16-118 6,PUBLIC URINATION 1ST OFFENSE,75.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 2969.0 14.0 492 SCHOLES STREET BROOKL...,MANHATTAN 235 WEST 48 STREET NEW YORK 10036 NE...,1,10036
182487,045180234Y,2019-02-27,SANITATION POLICE,UNKNOWN,BROOKLYN PARK TERRACE LLC,BROOKLYN,5125.0,47.0,2108,REGENT PLACE,BROOKLYN,11226,NEW YORK,BROOKLYN,2108,REGENT PLACE,BROOKLYN,11226,NEW YORK,MANHATTAN,100.0,0.0,AS06,A.C. 16-118 2 A,DIRTY SIDEWALK DIRTY AREA,100.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 5125.0 47.0 2108 REGENT PLACE BROOKLY...,BROOKLYN 2108 REGENT PLACE BROOKLYN 11226 NEW ...,3,11226
132816,0130348855,2002-08-03,SANITATION POLICE,ELIAS,RODITIS,QUEENS,2015.0,6.0,111-12,42 AVENUE,QUEENS,11368,NEW YORK,QUEENS,111-12,42 AVENUE,QUEENS,11368,NEW YORK,UNKNOWN,300.0,UNKNOWN,AS06,16-118 2,DIRTY SIDEWALK,50.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,QUEENS 2015.0 6.0 111-12 42 AVENUE QUEENS 1136...,QUEENS 111-12 42 AVENUE QUEENS 11368 NEW YORK,0,11368
39483,0190757656,2016-08-20,POLICE DEPT,MD G,KHAN,MANHATTAN,1443.0,24.0,1267,1 AVENUE,NEW YORK,10065,UNKNOWN,QUEENS,42-25,LAYTON STREET,ELMHURST,11373,NEW YORK,SAU: MANH,UNKNOWN,UNKNOWN,AF08,17-314 A,FAIL TO PERMIT REGULAR INSPECTIONS,10000.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,MANHATTAN 1443.0 24.0 1267 1 AVENUE NEW YORK 1...,QUEENS 42-25 LAYTON STREET ELMHURST 11373 NEW ...,2,11373
54101,0130219468,2002-10-04,SANITATION POLICE,UNKNOWN,VIZA REALTY CORP,STATEN IS,638.0,47.0,34,HARDY STREET,STATEN IS,10304,NEW YOR

In [330]:
# get a list of the zipcodes that do not matche ZCTA and convert before mapping it onto the new column
ZiptoZcta_Crosswalk_2021_NYC_Not_Matched = ZiptoZcta_Crosswalk_2021_NYC[ZiptoZcta_Crosswalk_2021_NYC['zip_join_type'] == "Spatial join to ZCTA"]
ZiptoZcta_Crosswalk_2021_NYC_Not_Matched.head(2)

,ZIP_CODE,PO_NAME,STATE,ZIP_TYPE,ZCTA,zip_join_type,ZIP_CODE_NYC
4984,10041,New York,NY,Post Office or large volume customer,10004,Spatial join to ZCTA,NYC
4986,10045,New York,NY,Post Office or large volume customer,10005,Spatial join to ZCTA,NYC


In [331]:
ZiptoZcta_Crosswalk_2021_NYC_Not_Matched_dict = dict(zip(ZiptoZcta_Crosswalk_2021_NYC_Not_Matched.ZIP_CODE, ZiptoZcta_Crosswalk_2021_NYC_Not_Matched.ZCTA))

In [332]:
ZiptoZcta_Crosswalk_2021_NYC_Not_Matched_dict

{'10041': '10004',
 '10045': '10005',
 '10055': '10022',
 '10060': '10199',
 '10090': '10007',
 '10104': '10019',
 '10105': '10019',
 '10106': '10019',
 '10107': '10019',
 '10118': '10001',
 '10120': '10001',
 '10121': '10011',
 '10122': '10001',
 '10123': '10001',
 '10151': '10022',
 '10155': '10022',
 '10158': '10016',
 '10166': '10017',
 '10175': '10173',
 '10176': '10017',
 '10178': '10016',
 '10270': '10005',
 '10281': '10280',
 '11241': '11201',
 '11242': '11201',
 '11243': '11217',
 '11249': '11211',
 '11252': '11209',
 '11256': '11208'}

In [324]:
#if the zip_code in the main df is in the ZCTA zip_code column
# then add the ZCTA number to the Respondent ZCTA column

need_to_convert = list(ZiptoZcta_Crosswalk_2021_NYC["ZIP_CODE"][ZiptoZcta_Crosswalk_2021_NYC['zip_join_type'] == "Spatial join to ZCTA"])

In [337]:
df['Respondent ZCTA'] = df['Respondent Address (Zip Code)'].map(ZiptoZcta_Crosswalk_2021_NYC_Not_Matched_dict)

In [340]:
df.sample(25)

,Ticket Number,Violation Date,Issuing Agency,Respondent First Name,Respondent Last Name,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),Violation Location (Street Name),Violation Location (City),Violation Location (Zip Code),Violation Location (State Name),Respondent Address (Borough),Respondent Address (House #),Respondent Address (Street Name),Respondent Address (City),Respondent Address (Zip Code),Respondent Address (State Name),Decision Location (Borough),Penalty Imposed,Paid Amount,Charge #1: Code,Charge #1: Code Section,Charge #1: Code Description,Charge #1: Infraction Amount,Charge #2: Code,Charge #2: Code Section,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,complete violation location,complete respondent location,Hearing Result,Respondent ZCTA
101132,0199042672,2019-11-07,POLICE DEPARTMENT,CIRINO,GALVEZ JERONIMO,QUEENS,1598.0,12.0,41-18,JUNCTION BOULEVARD,CORONA,11368,NEW YORK,MANHATTAN,609,WEST 189 STREET,NEW YORK,10040,NEW YORK,UNKNOWN,25.0,0.0,AX25,AC 10-125,OPEN CONTAINER CONSUMPTION OF ALCOHOL ON STREETS,25.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,QUEENS 1598.0 12.0 41-18 JUNCTION BOULEVARD CO...,MANHATTAN 609 WEST 189 STREET NEW YORK 10040 N...,1,NaN
157047,0182819744,2014-05-17,POLICE DEPARTMENT,FRANCISCO,PENA,BROOKLYN,815.0,46.0,5324,5 AVENUE,BROOKLYN,11220,NEW YORK,BROOKLYN,353,57 STREET,BROOKLYN,11220,NEW YORK,BROOKLYN,1000.0,0.0,AF01,A.C. 17-307 A,UNLICENSED MOBILE FOOD VENDOR,1000.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 815.0 46.0 5324 5 AVENUE BROOKLYN 112...,BROOKLYN 353 57 STREET BROOKLYN 11220 NEW YORK,3,NaN
41907,042515327X,2013-11-01,SANITATION POLICE,WANDA,STALLINGS,BRONX,2952.0,153.0,816,EAST 175 STREET,BRONX,10460,NEW YORK,BRONX,816,EAST 175 STREET,BRONX,10460,NEW YORK,UNKNOWN,300.0,0.0,AS6M,A.C. 16-118 2 A,DIRTY AREA,100.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BRONX 2952.0 153.0 816 EAST 175 STREET BRONX 1...,BRONX 816 EAST 175 STREET BRONX 10460 NEW YORK,1,NaN
23927,042701785M,2011-03-07,SANITATION POLICE,RENE,PENTOJA,QUEENS,3234.0,173.0,70-20,AUSTIN STREET,FOREST HILLS,11375,NEW YORK,QUEENS,61-18,GATES AVENUE,RIDGEWOOD,11385,NEW YORK,QUEENS,UNKNOWN,UNKNOWN,AS03,A.C. 16-118 1,LITTERING,100.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,QUEENS 3234.0 173.0 70-20 AUSTIN STREET FOREST...,QUEENS 61-18 GATES AVENUE RIDGEWOOD 11385 NEW ...,2,NaN
62212,0144086681,2004-10-19,SANITATION POLICE,UNKNOWN,RICH AUTO REPAIR,QUEENS,1824.0,53.0,126-15,37 AVENUE,QUEENS,11368,NEW YORK,QUEENS,126-15,37 AVENUE,QUEENS,11368,NEW YORK,UNKNOWN,300.0,357.0,AS16,A.C. 16-120(A),MAINTAINING RECEPTACLES,100.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,QUEENS 1824.0 53.0 126-15 37 AVENUE QUEENS 113...,QUEENS 126-15 37 AVENUE QUEENS 11368 NEW YORK,1,NaN
149310,0169564268,2009-06-03,NYPD TRANSPORT INTELLIGENCE DI,UNKNOWN,SANITATION SALVAGE CORP,MANHATTAN,1548.0,35.0,UNKNOWN,EAST 86 STREET,NEW YORK,10028,NEW YORK,BRONX,421,MANIDA STREET,BRONX,10474,NEW YORK,MANHATTAN,UNKNOWN,UNKNOWN,AD17,A.C. 19-123,NO STREET PROTECTION UNDER COMMERCIAL REFUSE C...,250.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,MANHATTAN 1548.0 35.0 EAST 86 STREET NEW YORK...,BRONX 421 MANIDA STREET BRONX 10474 NEW YORK,2,NaN
56534,0176473038,2011-01-02,POLICE DEPARTMENT,GAMAL,IBRAHIM,MANHATTAN,783.0,34.0,430,7 AVENUE,NEW YORK,10001,NEW YORK,BROOKLYN,1041,BUSHWICK AVENUE,BROOKLYN,11221,NEW YORK,UNKNOWN,1000.0,0.0,AF18,17-315 E,"IN BUS STOP, OR 10 FT. OF DRIVE, SUBWAY, CROSS...",100.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,MANHATTAN 783.0 34.0 430 7 AVENUE NEW YORK 100...,BROOKLYN 1041 BUSHWICK AVENUE BROOKLYN 11221 N...,0,NaN
136722,042505505K,2014-09-21,SANITATION POLICE,R THE,BANK OF NEW YORK AS TRUSTEE FO,BRONX,29

In [ ]:
df['Respondent ZCTA'] = np.where(df['Respondent Address (Zip Code)'].isin(nyc_zip_code_list), df['Respondent Address (Zip Code)'], "Not Matched")

In [ ]:
city_url =[]

for x in zip_codes:
    city_url.append('https://' + city +'.org/d/motorcycles-scooters/search/mca')
    
city_url